# 99_Coverage_Report

In [ ]:
import json, platform, sys
from pathlib import Path
import numpy as np
import pandas as pd

def find_repo_root() -> Path:
    p = Path.cwd().resolve()
    for cand in [p, *p.parents]:
        if (cand / 'inventory_pandas_api.json').exists():
            return cand
    raise FileNotFoundError('inventory_pandas_api.json not found from cwd upward')

ROOT = find_repo_root()
print('ROOT =', ROOT)
print('pandas:', pd.__version__)
print('python:', sys.version.split()[0])
print('platform:', platform.platform())
if not pd.__version__.startswith('2.2.'):
    print('WARNING: runtime pandas differs from target 2.2.*')


In [ ]:
inventory = json.loads((ROOT / 'inventory_pandas_api.json').read_text(encoding='utf-8'))
coverage_map = {
 'DataFrame':'coverage_dataframe.json','Series':'coverage_series.json','Index':'coverage_index.json',
 'GroupBy':'coverage_groupby.json','Window':'coverage_window.json','TopLevel':'coverage_toplevel.json'}
all_missing = {}
for t in inventory['meta']['target_order']:
    inv = inventory['targets'][t]
    inv_set = {r['qualname'] for r in inv}
    aliases = [r['qualname'] for r in inv if r.get('alias_of')]
    deprecated = [r['qualname'] for r in inv if r.get('deprecated') is True]
    cpath = ROOT / coverage_map[t]
    cov = json.loads(cpath.read_text(encoding='utf-8')) if cpath.exists() else {'covered_members':[],'skipped_members':{}}
    covered = set(cov.get('covered_members', []))
    skipped = cov.get('skipped_members', {})
    skipped_missing_reason = sorted([k for k,v in skipped.items() if not v])
    missing = sorted(inv_set - (covered | set(skipped.keys())))
    all_missing[t] = missing
    print(f'\n=== {t} ===')
    print('inventariati:', len(inv_set))
    print('covered:', len(covered))
    print('skipped:', len(skipped))
    print('missing:', len(missing))
    print('alias:', len(aliases))
    print('deprecated:', len(deprecated))
    if skipped_missing_reason:
        print('skipped senza reason:', skipped_missing_reason[:5])
success = all(len(v)==0 for v in all_missing.values())
print('\nSUCCESS=', success)